In [ ]:
import pandas as pd
import json
import glob
import os
import cv2
from tqdm import tqdm

In [ ]:
# TARGET_PATH = "/home/hemyo/Desktop/1_data/multi-keypoint-proto/temp/"
TARGET_PATH = "/run/user/1000/gvfs/smb-share:server=10.50.1.4,share=datalakev1/"
TARGET_PATH += "02_Salmon/01_Raw_data/02_keypoint_cropped_fish/2021(legacy-2kpt)/2_kpt_20210721_Huon_AM100_sample/"
TARGET_PATH += "4kpt_done/"

In [ ]:
belly_fin = ["belly_fin", "belly fin", "fin_belly", "fin belly", "belly-fin", "bell"]
back_fin = ["fin_top", "fin top", "top_fin", "top fin", "back fin", "back_fin", "top", "high"]
tail = ["tail center", "tail"]
nose = ["nose"]

def is_in_any_list(input_str: str) -> bool:
    return input_str in belly_fin or input_str in back_fin or input_str in tail or input_str in nose

In [ ]:
def convert_json_to_csv(filename, out):
    s = json.load(open(filename, 'r'))
    filename_base = s['imagePath']
    filepath = os.path.join(TARGET_PATH,filename_base)
    im = cv2.imread(filepath)
    h,w,c = im.shape
    
    # init multiple varialbes to 0
    (nose_x,nose_y,nose_visibility,
    eye_x,eye_y,eye_visibility,
    midfin_x,midfin_y,midfin_visibility,
    bellyfin_x,bellyfin_y,bellyfin_visibility,
    backfin_x,backfin_y,backfin_visibility,
    tailtip_x,tailtip_y,tailtip_visibility,
    tailtip2_x,tailtip2_y,tailtip2_visibility,
    tail_x,tail_y,tail_visibility) = (0,)*24

    for ann in s['shapes']:
        label = ann['label']
        x = int(ann['points'][0][0])
        y = int(ann['points'][0][1])
        # coordinates per label 
        if label in nose : nose_x = x ; nose_y = y; nose_visibility = 1
        if label in tail : tail_x = x ; tail_y = y; tail_visibility = 1
        if label in belly_fin : bellyfin_x = x ; bellyfin_y = y; bellyfin_visibility = 1
        if label in back_fin : backfin_x = x ; backfin_y = y; backfin_visibility = 1
        if not is_in_any_list(label) : 
            print(label,"is not in the any of the list ",filename_base)
        # if label == 'mid fin': midfin_x = x ; midfin_y = y; midfin_visibility = 1
        # if label == 'tail tip': 
        #     if tailtip_visibility == 0 : tailtip_x = x ; tailtip_y = y; tailtip_visibility = 1
        #     if tailtip_visibility == 1 : tailtip2_x = x ; tailtip2_y = y; tailtip2_visibility = 1 # second tail tip
        # if label == 'tail center': tail_x = x ; tail_y = y; tail_visibility = 1

    # retrun if missing label
    # if eye_visibility+nose_visibility+tail_visibility+bellyfin_visibility+backfin_visibility != 5 : return 0

    # write: fname, class, bbox_x1, bbox_y1, bbox_x2, bbox_y2
    out.write(f'{filename_base},0,0,0,{w},{h}')
    
    # write the extracted result
    out.write(f',{nose_x},{nose_y},{nose_visibility}')
    # out.write(f',{eye_x},{eye_y},{eye_visibility}')
    out.write(f',{tail_x},{tail_y},{tail_visibility}')
    if len(s['shapes']) == 4:
        out.write(f',{bellyfin_x},{bellyfin_y},{bellyfin_visibility}')
        out.write(f',{backfin_x},{backfin_y},{backfin_visibility}')
    
    # out.write(f',{midfin_x},{midfin_y},{midfin_visibility}')
    # out.write(f',{tailtip_x},{tailtip_y},{tailtip_visibility}')
    # out.write(f',{tailtip2_x},{tailtip2_y},{tailtip2_visibility}')
    
    
    out.write('\n')

In [ ]:
# define save csv file
out_file = os.path.normpath(TARGET_PATH) + '.csv'
out = open(out_file, 'w')

csv_head = """filename,class,bbox_x1,bbox_y1,bbox_x2,bbox_y2,
            nose_x,nose_y,nose_visibility,
            tail_x,tail_y,tail_visibility,
            bellyfin_x,bellyfin_y,bellyfin_visibility,
            backfin_x,backfin_y,backfin_visibility
            """.replace('\n',"").replace('\ ',"").replace(' ',"")
# csv_head = "asd"
out.write(csv_head+"\n")

json_list = glob.glob(TARGET_PATH+"/*.json")
print(len(json_list), "json files are globbed")

# convert json to csv
for filename in tqdm(json_list):
    convert_json_to_csv(filename, out)

out.close()
print("done")

In [ ]:
filename

In [ ]:
png_list = glob.glob(TARGET_PATH+"/*.png")
for png_f in sorted(png_list) :
    json_f = png_f.replace("png","json")
    if not os.path.isfile(json_f) : 
        print(png_f)
        os.remove(png_f)
    # break